In [6]:
import mido
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np

Using TensorFlow backend.


In [11]:
import os

filename = '../data/kpop_midi_samples'
filenames = os.listdir(filename)
filenames = [i for i in filenames if i[-3:] == 'mid']
kpop_files  = [MidiFile(filename+'/'+i) for i in filenames] 


In [15]:
lst_note = []
for song in kpop_files:
    for note in song: 
        if not note.is_meta and note.type == 'note_on' and note.channel == 0 :
            nb = note.bytes()
            lst_note.append(nb[1])

In [18]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(np.array(lst_note).reshape(-1,1))
lst_notes = list(scaler.transform(np.array(lst_note).reshape(-1,1)))

In [19]:
# LSTM layers 
notes = [list(note) for note in lst_notes]

x = []
y = []
# batch note #
prev = 30
for i in range(len(notes)- prev):
    x.append(lst_notes[i:i+ prev])
    y.append(lst_notes[i+ prev])

x_test = y[-300:]
x = x[:-300]
y = y[:-300]

In [24]:
#train model 
model = Sequential()
model.add(LSTM(256, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(Dropout(0.6))
model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dropout(0.6))
model.add(Dense(1))
model.add(Activation('linear'))
optimizer = Adam(lr=0.001)
model.compile(loss='mse', optimizer=optimizer)
filepath="./Checkpoints/checkpoint_model_{epoch:02d}.hdf5"
model_save_callback = ModelCheckpoint(filepath, monitor='val_acc', 
                                      verbose=1, save_best_only=False, 
                                      mode='auto', period=5)

model.fit(np.array(X), np.array(y), 32, 5, verbose=1, callbacks=[model_save_callback])

Epoch 1/5
24112/24112 [==============================] - 82s 3ms/step - loss: 0.0244
Epoch 2/5
24112/24112 [==============================] - 75s 3ms/step - loss: 0.0200
Epoch 3/5
24112/24112 [==============================] - 76s 3ms/step - loss: 0.0186
Epoch 4/5
24112/24112 [==============================] - 77s 3ms/step - loss: 0.0182
Epoch 5/5
24112/24112 [==============================] - 78s 3ms/step - loss: 0.0179

Epoch 00005: saving model to ./Checkpoints/checkpoint_model_05.hdf5


In [25]:
prediction = model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]

In [26]:
#make new file

mid = MidiFile()
track = MidiTrack()
i = 0
for note in prediction:
    msg = Message.from_bytes(bytes[0:3])
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    i += 1
    msg.time = i
    track.append(msg)
mid.tracks.append(track)
mid.save('kpop_piano_generation.mid')